In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers, Model
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class GRANXModel(tf.keras.Model):    
    def __init__(self, sequence_length, n_features, hidden_units=64, attention_heads=8, 
                 correction_rate=0.01, dropout_rate=0.2):
        super(GRANXModel, self).__init__()
        
        self.sequence_length = sequence_length
        self.n_features = n_features
        self.hidden_units = hidden_units
        self.attention_heads = attention_heads
        self.correction_rate = correction_rate
        self.conv1d_layers = [
            layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'),
            layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Dropout(dropout_rate)
        ]
        self.temporal_layers = [
            layers.GRU(hidden_units, return_sequences=True, dropout=dropout_rate),
            layers.LSTM(hidden_units, return_sequences=True, dropout=dropout_rate),
            layers.BatchNormalization()
        ]
        self.attention = layers.MultiHeadAttention(
            num_heads=attention_heads, 
            key_dim=hidden_units,
            dropout=dropout_rate
        )
        self.feature_dense = layers.Dense(hidden_units, activation='tanh')
        self.feature_norm = layers.LayerNormalization()
        self.correction_dense = layers.Dense(hidden_units, activation='linear')
        self.output_projection = layers.Dense(1, activation='linear')
        self.global_pool = layers.GlobalAveragePooling1D()
        self.final_dropout = layers.Dropout(dropout_rate)
    
    def compute_feature_maps(self, x):
        for layer in self.conv1d_layers:
            x = layer(x)
        for layer in self.temporal_layers:
            x = layer(x)
        M_t = self.feature_dense(x)
        M_t = self.feature_norm(M_t)
        return M_t
    
    def compute_attention_weights(self, M_t):
        attended_features, attention_weights = self.attention(
            M_t, M_t, return_attention_scores=True
        )
        alpha_t = tf.nn.softmax(tf.reduce_mean(attention_weights, axis=1), axis=-1)
        return attended_features, alpha_t
    
    def compute_gradient_correction(self, weighted_features, y_true=None):
        if y_true is not None and self.training:
            with tf.GradientTape() as tape:
                tape.watch(weighted_features)
                temp_pred = self.output_projection(self.global_pool(weighted_features))
                loss = tf.reduce_mean(tf.square(temp_pred - y_true))
            gradients = tape.gradient(loss, weighted_features)
            correction = -self.correction_rate * gradients if gradients is not None else tf.zeros_like(weighted_features)
        else:
            correction = -self.correction_rate * self.correction_dense(weighted_features)
        return correction
    
    def call(self, inputs, training=None, y_true=None):
        M_t = self.compute_feature_maps(inputs)
        attended_features, alpha_t = self.compute_attention_weights(M_t)
        alpha_expanded = tf.expand_dims(alpha_t, axis=-1)
        weighted_sum = weighted_sum = tf.reduce_sum(attended_features * tf.reduce_mean(alpha_expanded, axis=2), axis=1)
        correction = self.compute_gradient_correction(attended_features, y_true)
        corrected_features = attended_features - correction
        pooled_features = self.global_pool(corrected_features)
        pooled_features = self.final_dropout(pooled_features, training=training)
        output = self.output_projection(pooled_features)
        return output


In [ ]:
class EnergyDataProcessor:
    """Data processor for household energy consumption data"""
    
    def __init__(self, sequence_length=24, prediction_horizon=1):
        self.sequence_length = sequence_length
        self.prediction_horizon = prediction_horizon
        self.scaler = StandardScaler()
        self.target_scaler = MinMaxScaler()
        
    def load_smart_meter_data(self, file_path):
        try:
            data = pd.read_csv(file_path)
            data = data.replace(-1, np.nan)
            data = data.interpolate(method='linear')
            return data
        except Exception as e:
            print(f"Error loading data: {e}")
            return None
    
    def aggregate_to_hourly(self, data):
        if len(data) == 86400:
            hourly_data = data.values.reshape(24, 3600, -1).mean(axis=1)
            return pd.DataFrame(hourly_data, columns=data.columns)
        return data
    
    def create_sequences(self, data, target_column='powerallphases'):
        feature_columns = [
            'powerallphases', 'powerl1', 'powerl2', 'powerl3',
            'currentneutral', 'currentl1', 'currentl2', 'currentl3',
            'voltagel1', 'voltagel2', 'voltagel3'
        ]
        available_columns = [col for col in feature_columns if col in data.columns]
        features = data[available_columns].values
        target = data[target_column].values
        
        features_scaled = self.scaler.fit_transform(features)
        target_scaled = self.target_scaler.fit_transform(target.reshape(-1, 1)).flatten()
        
        X, y = [], []
        for i in range(len(features_scaled) - self.sequence_length - self.prediction_horizon + 1):
            X.append(features_scaled[i:(i + self.sequence_length)])
            y.append(target_scaled[i + self.sequence_length + self.prediction_horizon - 1])
        return np.array(X), np.array(y)
    
    def create_synthetic_data(self, n_samples=1000):
        np.random.seed(42)
        hours = np.arange(n_samples) % 24
        days = np.arange(n_samples) // 24
        base_consumption = (
            500 +
            200 * np.sin(2 * np.pi * hours / 24) +
            100 * np.sin(2 * np.pi * days / 7) +
            50 * np.random.normal(0, 1, n_samples)
        )
        features = {
            'powerallphases': base_consumption,
            'powerl1': base_consumption * 0.4 + np.random.normal(0, 20, n_samples),
            'powerl2': base_consumption * 0.3 + np.random.normal(0, 15, n_samples),
            'powerl3': base_consumption * 0.3 + np.random.normal(0, 15, n_samples),
            'currentneutral': base_consumption * 0.02 + np.random.normal(0, 1, n_samples),
            'currentl1': base_consumption * 0.008 + np.random.normal(0, 0.5, n_samples),
            'currentl2': base_consumption * 0.006 + np.random.normal(0, 0.5, n_samples),
            'currentl3': base_consumption * 0.006 + np.random.normal(0, 0.5, n_samples),
            'voltagel1': 230 + np.random.normal(0, 5, n_samples),
            'voltagel2': 230 + np.random.normal(0, 5, n_samples),
            'voltagel3': 230 + np.random.normal(0, 5, n_samples),
        }
        return pd.DataFrame(features)


In [ ]:
def train_granx_model(X_train, y_train, X_val, y_val, epochs=100, batch_size=32, learning_rate=0.001):
    model = GRANXModel(
        sequence_length=X_train.shape[1],
        n_features=X_train.shape[2],
        hidden_units=64,
        attention_heads=8,
        correction_rate=0.01
    )
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mse',
        metrics=['mae']
    )
    callbacks = [
        keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(patience=10, factor=0.5, min_lr=1e-6),
        keras.callbacks.ModelCheckpoint('granx_model.h5', save_best_only=True)
    ]
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=1
    )
    return model, history


In [ ]:
def evaluate_model(model, X_test, y_test, target_scaler):
    y_pred = model.predict(X_test)
    y_test_original = target_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
    y_pred_original = target_scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
    
    mse = mean_squared_error(y_test_original, y_pred_original)
    mae = mean_absolute_error(y_test_original, y_pred_original)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test_original, y_pred_original)
    
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"MSE: {mse:.2f}")
    print(f"R²: {r2:.4f}")
    
    return {
        'rmse': rmse,
        'mae': mae,
        'mse': mse,
        'r2': r2,
        'y_true': y_test_original,
        'y_pred': y_pred_original
    }

def plot_results(results, history):
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    axes[0, 0].plot(history.history['loss'], label='Training Loss')
    axes[0, 0].plot(history.history['val_loss'], label='Validation Loss')
    axes[0, 0].set_title('Model Loss'); axes[0, 0].legend()
    
    axes[0, 1].plot(history.history['mae'], label='Training MAE')
    axes[0, 1].plot(history.history['val_mae'], label='Validation MAE')
    axes[0, 1].set_title('Model MAE'); axes[0, 1].legend()
    
    axes[1, 0].scatter(results['y_true'], results['y_pred'], alpha=0.6)
    axes[1, 0].plot([results['y_true'].min(), results['y_true'].max()],
                    [results['y_true'].min(), results['y_true'].max()], 'r--')
    axes[1, 0].set_title('Predictions vs Actual')
    
    n_points = min(200, len(results['y_true']))
    axes[1, 1].plot(results['y_true'][:n_points], label='Actual')
    axes[1, 1].plot(results['y_pred'][:n_points], label='Predicted')
    axes[1, 1].set_title('Time Series Comparison')
    axes[1, 1].legend()
    plt.tight_layout()
    plt.show()


In [ ]:
def main():
    print("GRAN-X Energy Forecasting Model")
    print("="*50)
    
    processor = EnergyDataProcessor(sequence_length=24, prediction_horizon=1)
    print("Creating synthetic energy consumption data...")
    data = processor.create_synthetic_data(n_samples=2000)
    
    print("Creating sequences for training...")
    X, y = processor.create_sequences(data)
    
    split_idx = int(0.7 * len(X))
    val_idx = int(0.85 * len(X))
    X_train, y_train = X[:split_idx], y[:split_idx]
    X_val, y_val = X[split_idx:val_idx], y[split_idx:val_idx]
    X_test, y_test = X[val_idx:], y[val_idx:]
    
    print(f"Training set: {X_train.shape}")
    print(f"Validation set: {X_val.shape}")
    print(f"Test set: {X_test.shape}")
    
    print("\nTraining GRAN-X model...")
    model, history = train_granx_model(X_train, y_train, X_val, y_val, epochs=50, batch_size=32)
    
    print("\nEvaluating model...")
    results = evaluate_model(model, X_test, y_test, processor.target_scaler)
    
    plot_results(results, history)
    print("\nGRAN-X model training completed successfully!")
    return model, results, processor

model, results, processor = main()
